# Pull ESPN Ownership Data

### MSP 9.4.2017


In [41]:
from IPython.display import HTML
HTML('''<script>code_show=true; function code_toggle() { if (code_show){ $('div.input').hide();} else { $('div.input').show(); } code_show = !code_show} $( document ).ready(code_toggle);</script><form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [14]:
# boilerplate imports

import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import matplotlib as mpl
cmap = cm.gnuplot




In [1]:
# imports for scraping

from bs4 import BeautifulSoup

import requests

import operator



### Change In Ownership

Also set up an archive.

In [109]:

Ownership = {}


for tablenum in range(0,13):

    # cycle through all positions
    r  = requests.get('http://games.espn.com/flb/addeddropped?slotCategoryId='+str(tablenum))

    data = r.text

    soup = BeautifulSoup(data)

    tables = soup.findAll('table')


    for table in tables: 

        headings = [th.get_text() for th in table.find("tr").find_all("th")]
        #print headings


        for row in table.find_all("tr")[1:]: 

        #    sav = [td.find('a') for td in row.find_all("td")]
            sav = [td.get_text() for td in row.find_all("td")]

            w = sav.index(u'\xa0')

            #print (sav[1].split(',')[0]).strip('*')

            try:
                # block out all non-rankings
                xx = float(sav[0])

                #print sav[0:w]
                #print sav[w+1:len(sav)]

                # added
                Ownership[(sav[1].split(',')[0]).strip('*')] = float(sav[5])

                #dropped
                Ownership[(sav[w+2].split(',')[0]).strip('*')] = float(sav[w+6])

            except:
                pass

            #[u'RK', u'PLAYER, TEAM', u'POS', u'LAST', u'CUR', u'7-DAY +/-']
        

today = datetime.date.today()

f = open('ownership-archive/'+str(today)+'.txt','w')

for index,key in enumerate(Ownership.keys()):
    print >>f,key,Ownership[key]
    
f.close()

In [112]:
print l[0]

positives = ['congratulates']

print np.sum(np.array([(x in l[0]) for x in positives]))



Minnesota Twins' Byron Buxton, right, congratulates Brian Dozier as he scores on a double by Joe Mauer off Kansas City Royals pitcher Onelki ...
1


In [115]:
negatives = ['rehab','injure','disable','DL','suspension','surgery']

#[(x in string_in) for x in negatives]

def check_words(string_in,words):
    total = [(x in string_in) for x in words]
    return np.sum(np.array(total))
    

for index,key in enumerate(Ownership.keys()):
    if Ownership[key] < -5.:
        print key
        print '--------------'
        l = scrape_news_summaries(key)
        print l[0]
        print check_words(l[0],negatives),' ',
        print check_words(l[1],negatives),' ',
        print check_words(l[2],negatives)
        print ''



Evan Gattis
--------------
Gattis (wrist) is scheduled to report to Low-A Quad Cities in the next couple of days to begin a rehab assignment, Brian McTaggart of MLB.com ...
1   1   1

Tyler Flowers
--------------
The Atlanta Braves have placed catcher Tyler Flowers on the 10-day disabled ... Flowers suffered the injury during Monday's game against the Phillies when he ...
1   1   2

Brandon Belt
--------------
Manager Bruce Bochy remains hopeful that Belt will return this season, but acknowledged Tuesday that he may not return until 2018.
0   0   1

Jedd Gyorko
--------------
LOUIS -- Jedd Gyorko landed on the 10-day disabled list with a right hamstring strain on Sunday, but the Cardinals are optimistic that Gyorko will return to the ...
1   0   0

Carlos Gonzalez
--------------
Carlos Gonzalez #5 of the Colorado Rockies celebrates after walking in the ninth inning with the bases loaded that gave the Rockies the 4-3 ...
0   0   0

Miguel Cabrera
--------------
DETROIT -- Tigers star Mi

In [108]:
for index,key in enumerate(Ownership.keys()):
    if Ownership[key] > 5.:
        print key
        print '--------------'
        l = scrape_news_summaries(key)
        print l[0]
        print ''




Rhys Hoskins
--------------
Philadelphia's Rhys Hoskins hit 11 home runs in just 18 career games, reaching that total faster than any player in major league history. “It's just ...

Shin-Soo Choo
--------------
Choo is out of the lineup Sunday against the Angels. Choo's absence will indirectly open up a start at first base for Will Middlebrooks, who will ...

Robinson Chirinos
--------------
Chirinos and leadoff man Delino DeShields supplied back-to-back home runs off Andrew Heaney in the fourth inning, knocking the Angels ...

Yolmer Sanchez
--------------
Sanchez has started 18 consecutive games due to injuries to several infielders. He's hitting .343 (23-for-67) with four homers and 14 RBI during ...

Jorge Polanco
--------------
Jorge Polanco, SS, Twins: In case you missed Polanco's impressive impersonation of ex-teammate Eduardo Nunez last month, all you did was ...

Byron Buxton
--------------
MINNEAPOLIS – Byron Buxton put on a show in the final month of the season for a non-com

In [51]:
print Ownership['Justin Verlander']

KeyError: 'Justin Verlander'

### News Alerts

Another interesting thing is grabbing Google News findings on players to see if we can determine what the narrative leading to the explosion in pickups or drops is coming from.

Good keywords:
1. Trade
2. Hot
3. Streak
4. Fire
5. Homer


Bad keywords:
1. Cold
2. Bench
3. Disabled/DL
4. Injure
5. Suspension



In [106]:
#https://stackoverflow.com/questions/39354587/scraping-google-news-with-beautifulsoup-returns-empty-results

from bs4 import BeautifulSoup
import requests
import time
from random import randint


def scrape_news_summaries(s):
    time.sleep(randint(0, 2))  # relax and don't let google be angry
    
    r = requests.get("http://www.google.com/search?q="+s+"&tbm=nws")    
    # Print the status code
    #print(r.status_code)  
    
    content = r.text
    #print content
    
    news_summaries = []
    soup = BeautifulSoup(content, "html.parser")
    #st_divs = soup.find_all("div", class_="gs-info")
    st_divs = soup.findAll("div", {"class": "st"})
    #st_divs = soup.findAll("div", {"class": "rc"})


    for st_div in st_divs:
        #print st_div
        news_summaries.append(st_div.text)
    return news_summaries




#http://www.espn.com/search/results?q=Alex+Wood
#r = requests.get("http://www.google.com/search?q="+s+"&tbm=nws")

l = scrape_news_summaries("Justin+Verlander")

for n in l:
    print(n)
    print('')

Waking up on Friday morning and seeing that Justin Verlander had been traded hit us all like a ton of bricks. Despite the majority of Bless You ...

Detroit Tigers general manager Al Avila scouted Justin Verlander in the spring of 2004, when Verlander pitched for Old Dominion on a day that ...

I'm among the last of us here at BYB to truly weigh in on the Justin Verlander trade. Most of the staff members have publicized their feelings, ...

"On behalf of the Detroit Tigers, we sincerely thank Justin Verlander for his remarkable 13 seasons of dedication to the organization," Tigers ...

... when he found out that his team had acquired Justin Verlander, one of baseball's most decorated pitchers, in a trade with the Detroit Tigers.

Fans of Justin Verlander woke up to some jolting news Friday morning: The ace pitcher was traded to the Houston Astros for three prospects.

The Houston Astros have acquired Justin Verlander in a trade with the ... The 34-year-old Verlander, who won the Cy You

### Stats Alerts

We should also scrape the fangraphs gamelog page to see if there is anything obvious there.

### Crazy

It would be awesome to try and make a model to predict ownership data based on news stories.

In [63]:
r  = requests.get('http://www.espn.com/mlb/player/_/id/33095/jose-abreu')

data = r.text

soup = BeautifulSoup(data)

fantasy_table = soup.find_all('div', id='fantasy-content')

print fantasy_table#.find("tr")

#for table in tables: 

#        headings = [th.get_text() for th in table.find("tr").find_all("th")]
#        #print headings

#<div class="mod-content" id="fantasy-content">


#(row in table.find_all("tr", class_="grid_postseason"))

[<div class="mod-content" id="fantasy-content">{"playerId":5600,"percentChange":-0.2,"averageDraftPosition":46.9,"percentOwned":96.8,"playerRaterSEASON":7.2,"playerRater7DAY":2.3,"mostRecentNews":{"news":"Abreu went 1-for-4 with a two-run home run during Sunday's win over Tampa Bay.","spin":"Abreu missed the previous three games with an elbow injury, but Sunday's blast affirmed his health. With a .301/.354/.540 slash line, 27 homers, 82 RBI and 72 runs for the campaign, the 30-year-old first baseman is in line to post another rock-solid showing. The supporting cast surrounding him in the White Sox's lineup limits his upside, but the slugger continues to move the fantasy needle nonetheless.","date":"Sun Sep 3"},"fullName":"Jose Abreu","seasonOutlook":{"outlook":"Though in many respects, Abreu's performance has regressed since he broke into the majors in 2014, let's not understate the historical significance of his first three big-league seasons: He, along with Joe DiMaggio and Albert Pu